# Taller ML grupo 02.
### Codigo e implementación https://github.com/Psychofun/Red-Neuronal-Numpy
Autor: PsyFun
##Preprocesamiento de los datos, para estandarizar ("gre","gpa") e individualizar la caracteristica categorica "rank"

#### Conversion de una variable categorica "rank" a variables binarias "rank_1","rank_2","rank_3", "rank_4"

In [ ]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv') #Lee csv
print(admissions.describe()) #Tabla
print(admissions.shape) #(numero de datos por columna, numero de columnas)
print(admissions.head()) #Primeros datos

            admit         gre         gpa       rank
count  400.000000  400.000000  400.000000  400.00000
mean     0.317500  587.700000    3.389900    2.48500
std      0.466087  115.516536    0.380567    0.94446
min      0.000000  220.000000    2.260000    1.00000
25%      0.000000  520.000000    3.130000    2.00000
50%      0.000000  580.000000    3.395000    2.00000
75%      1.000000  660.000000    3.670000    3.00000
max      1.000000  800.000000    4.000000    4.00000
(400, 4)
   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


In [ ]:
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1) #Agregar rank_i (i = 1,2,3,4)
print(data.describe())
data = data.drop('rank', axis=1) #Borrar 'rank'
print(data.shape)
print(data.head())

            admit         gre         gpa       rank      rank_1      rank_2  \
count  400.000000  400.000000  400.000000  400.00000  400.000000  400.000000   
mean     0.317500  587.700000    3.389900    2.48500    0.152500    0.377500   
std      0.466087  115.516536    0.380567    0.94446    0.359955    0.485369   
min      0.000000  220.000000    2.260000    1.00000    0.000000    0.000000   
25%      0.000000  520.000000    3.130000    2.00000    0.000000    0.000000   
50%      0.000000  580.000000    3.395000    2.00000    0.000000    0.000000   
75%      1.000000  660.000000    3.670000    3.00000    0.000000    1.000000   
max      1.000000  800.000000    4.000000    4.00000    1.000000    1.000000   

           rank_3      rank_4  
count  400.000000  400.000000  
mean     0.302500    0.167500  
std      0.459916    0.373889  
min      0.000000    0.000000  
25%      0.000000    0.000000  
50%      0.000000    0.000000  
75%      1.000000    0.000000  
max      1.000000    1.

Estandarizacion de las variables ("gre","gpa")

In [ ]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
print(data.shape)
print(data.describe())

(400, 7)
            admit           gre           gpa      rank_1      rank_2  \
count  400.000000  4.000000e+02  4.000000e+02  400.000000  400.000000   
mean     0.317500 -3.907985e-16  2.198242e-16    0.152500    0.377500   
std      0.466087  1.000000e+00  1.000000e+00    0.359955    0.485369   
min      0.000000 -3.183094e+00 -2.968993e+00    0.000000    0.000000   
25%      0.000000 -5.860633e-01 -6.829288e-01    0.000000    0.000000   
50%      0.000000 -6.665712e-02  1.340106e-02    0.000000    0.000000   
75%      1.000000  6.258844e-01  7.360075e-01    0.000000    1.000000   
max      1.000000  1.837832e+00  1.603135e+00    1.000000    1.000000   

           rank_3      rank_4  
count  400.000000  400.000000  
mean     0.302500    0.167500  
std      0.459916    0.373889  
min      0.000000    0.000000  
25%      0.000000    0.000000  
50%      0.000000    0.000000  
75%      1.000000    0.000000  
max      1.000000    1.000000  


Dividir el conjunto de datos para entrenamiento y evaluación del modelo

In [ ]:
print(data.index)

RangeIndex(start=0, stop=400, step=1)


In [ ]:
# Split off random 10% of the data for testing
np.random.seed(21)

sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
train_data, test_data = data.iloc[sample], data.drop(sample)
print(train_data.shape)
print(test_data.shape)

(360, 7)
(40, 7)


División de los datos en caracteristicas y etiquetas

In [ ]:
features, targets = train_data.drop('admit', axis=1), train_data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

Función de Activación

In [ ]:
def sigmoide(x): #O(1)
    return 1/(1 + np.exp(-x))

In [ ]:
# Hyperparameters
n_hidden = 2 # Número de unidades en la capa escondida - O(1)
epochs = 1000 # Número de iteraciones sobre el conjunto de entrenamiento - O(1)
alpha = 0.01 # Taza de aprendizaje - O(1)

ult_costo = None #O(1)

m,k = features.shape # Número de ejemplos de entrenamiento, número de dimensiones en los datos - O(1)
# Inicialización de los pesos
entrada_escondida = np.random.normal(scale = 1/k**0.5,size = (k,n_hidden)) #O(n)
escondida_salida = np.random.normal(scale = 1/k**0.5,size = n_hidden) #O(n)
print(entrada_escondida.shape)
print(escondida_salida.shape)
# Entrenamiento
for e in range(epochs): #O(n)
    # Variables para el gradiente
    gradiente_entrada_escondida = np.zeros(entrada_escondida.shape) #O(entrada_escondida.shape) = O(n)
    gradiente_escondida_salida =  np.zeros(escondida_salida.shape) #O(escondida_salida.shape) = O(n)
    # Itera sobre el conjunto de entrenamiento
    for x,y in zip(features.values,targets): #O(n)
        # Pasada hacia adelande (forward pass) or forward propagation
        z = sigmoide(np.matmul(x, entrada_escondida)) #O(n)
        y_ =sigmoide(np.matmul(escondida_salida,z)) # predicción - O(n)
        # Pasada hacia atrás (backward pass)
        salida_error = (y - y_) * y_ *(1- y_) #O(1)
        escondida_error = np.dot(salida_error, escondida_salida) * z * (1 -z) #O(n)

        gradiente_entrada_escondida += escondida_error * x[:,None] #O(1)
        gradiente_escondida_salida += salida_error * z #O(1)
    # Actualiza los parámetros (pesos)
    entrada_escondida += alpha * gradiente_entrada_escondida / m #O(1)
    escondida_salida +=  alpha * gradiente_escondida_salida / m #O(1)

    if e % (epochs / 10 ) == 0: #O(n)
        z = sigmoide(np.dot(features.values, entrada_escondida)) #O(n)
        y_ = sigmoide(np.dot(z, escondida_salida)) #O(n)

        # Función de costo
        costo = np.mean(( y_ - targets)**2 ) #O(n)

        if ult_costo  and ult_costo < costo: #O(1)
            print("Costo de  entrenamiento: ", costo, " ADVERTENCIA -  Costo subiendo")
        else:
            print("Costo de entrenamiento: ", costo )
        
        ult_costo = costo 

#  Precisión en los datos de prueba 
z = sigmoide(np.dot(features_test, entrada_escondida)) #O(n)
y_ = sigmoide(np.dot(z, escondida_salida)) #O(n)

predicciones =  y_ > 0.5 
precision = np.mean(predicciones == targets_test) #O(n)
print("Precisión: {:.3f}".format(precision)) #O(n)

#Asi el coste computacional de este entrenamiento es de O(n)




(6, 2)
(2,)
Costo de entrenamiento:  0.22749585224305116
Costo de entrenamiento:  0.2269025619637309
Costo de entrenamiento:  0.22633680141776774
Costo de entrenamiento:  0.22579695855377327
Costo de entrenamiento:  0.22528152075551738
Costo de entrenamiento:  0.22478906897245235
Costo de entrenamiento:  0.22431827212743183
Costo de entrenamiento:  0.2238678818008567
Costo de entrenamiento:  0.22343672718793942
Costo de entrenamiento:  0.2230237103237552
Precisión: 0.650


To train using only the necesary epochs.

In [ ]:
# Hyperparameters
n_hidden = 2 # Número de unidades en la capa escondida
epochs = 1000 # Número de iteraciones sobre el conjunto de entrenamiento
alpha = 0.01 # Taza de aprendizaje

ult_costo = None 

m,k = features.shape # Número de ejemplos de entrenamiento, número de dimensiones en los datos 
# Inicialización de los pesos
entrada_escondida = np.random.normal(scale = 1/k**0.5,size = (k,n_hidden))
escondida_salida = np.random.normal(scale = 1/k**0.5,size = n_hidden)
print(entrada_escondida.shape)
print(escondida_salida.shape)
# Entrenamiento
#for e in range(epochs):
precision_desired = 0.7
precision_prev = -1
precision_actual = 0
e = -1
while precision_actual < precision_desired and precision_actual != precision_prev:
    e += 1
    # Variables para el gradiente
    gradiente_entrada_escondida = np.zeros(entrada_escondida.shape)
    gradiente_escondida_salida =  np.zeros(escondida_salida.shape)
    # Itera sobre el conjunto de entrenamiento
    for x,y in zip(features.values,targets):
        # Pasada hacia adelande (forward pass) or forward propagation
        z = sigmoide(np.matmul(x, entrada_escondida))
        y_ =sigmoide(np.matmul(escondida_salida,z)) # predicción 
        # Pasada hacia atrás (backward pass)
        salida_error = (y - y_) * y_ *(1- y_)
        escondida_error = np.dot(salida_error, escondida_salida) * z * (1 -z)

        gradiente_entrada_escondida += escondida_error * x[:,None]
        gradiente_escondida_salida += salida_error * z 
    # Actualiza los parámetros (pesos)
    entrada_escondida += alpha * gradiente_entrada_escondida / m 
    escondida_salida +=  alpha * gradiente_escondida_salida / m 

    if e % (epochs / 10 ) == 0:
        z = sigmoide(np.dot(features.values, entrada_escondida))
        y_ = sigmoide(np.dot(z, escondida_salida))

        # Función de costo
        costo = np.mean(( y_ - targets)**2 )

        if ult_costo  and ult_costo < costo:
            print("Costo de  entrenamiento: ", costo, " ADVERTENCIA -  Costo subiendo")
        else:
            print("Costo de entrenamiento: ", costo, "Epoca", e, precision_actual)
        
        ult_costo = costo 
    z = sigmoide(np.dot(features_test, entrada_escondida))
    y_ = sigmoide(np.dot(z, escondida_salida))

    predicciones =  y_ > 0.5 
    precision_prev = precision_actual
    precision_actual = np.mean(predicciones == targets_test)

#  Precisión en los datos de prueba 
z = sigmoide(np.dot(features_test, entrada_escondida))
y_ = sigmoide(np.dot(z, escondida_salida))

predicciones =  y_ > 0.5 
precision = np.mean(predicciones == targets_test)
print("Precisión: {:.3f}".format(precision))
print("Numero de épocas requeridas", e)
if precision_actual == precision_prev:
    print("Bucle terminado debido a que la precisión no cambió")
else:
    print("La precisión alcanzó el valor deseado")



(6, 2)
(2,)
Costo de entrenamiento:  0.23096637380865542 Epoca 0 0
Precisión: 0.650
Numero de épocas requeridas 1
Bucle terminado debido a que la precisión no cambió
